In [10]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
import random
def generate(size=8, p=None):
    if p == None or p == 0:
        p = random.uniform(0.1, 1)
    valid = False
    def is_valid(arr, r=0, c=0):
        if arr[r][c] == 'G':
            return True
        
        tmp = arr[r][c]
        arr[r][c] = "#"
        
        if r+1 < size and arr[r+1][c] not in '#H':
            if is_valid(arr, r+1, c) == True:
                arr[r][c] = tmp
                return True
        
        if c+1 < size and arr[r][c+1] not in '#H':
            if is_valid(arr, r, c+1) == True:
                arr[r][c] = tmp
                return True
        
        if r-1 >= 0 and arr[r-1][c] not in '#H':
            if is_valid(arr, r-1, c) == True:
                arr[r][c] = tmp
                return True
        
        if c-1 >= 0 and arr[r][c-1] not in '#H':
            if is_valid(arr,r, c-1) == True:
                arr[r][c] = tmp
                return True
        arr[r][c] = tmp
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F','H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
        p *= 1.1
    return ["".join(x) for x in res]

In [24]:
#env = gym.make('FrozenLake-v0')

# Create a non-skid version of Frozen Lake
from gym.envs.registration import register, spec
from gym import envs

MY_ENV_NAME='FrozenLakeNonskid8x8-v0'

if MY_ENV_NAME in envs.registry.env_specs:
    envs.registry.env_specs.pop(MY_ENV_NAME)

register(
    id=MY_ENV_NAME,
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'is_slippery': True, 'desc': generate(size=5)},
    timestep_limit=100,
    reward_threshold=0.78, # optimum = .8196
)
env = gym.make(MY_ENV_NAME)
env.render()

[2018-01-21 21:57:17,501] Making new env: FrozenLakeNonskid8x8-v0



SFHFF
FFFFF
HFFFH
FFFFF
FFFFG


In [25]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8
y = .95
num_episodes = 2000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)


In [26]:
print(Q)

[[  1.92824538e-02   1.95491964e-02   2.64733040e-01   1.85338486e-02]
 [  3.20169907e-01   7.92810659e-05   6.60827602e-04   2.13108396e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  7.79113341e-03   3.70332856e-03   1.84587068e-01   2.74595876e-03]
 [  9.17509714e-02   1.29036482e-02   2.74595876e-03   3.51728706e-03]
 [  5.43479893e-04   6.26255506e-04   5.26758572e-03   3.43685505e-01]
 [  3.40846609e-03   3.23397391e-01   6.07806366e-03   1.62540912e-02]
 [  6.41444256e-04   5.62202420e-01   7.06901715e-04   2.47365199e-03]
 [  3.56109814e-01   4.49772639e-04   9.58861222e-03   4.88732926e-05]
 [  2.54483091e-04   5.01264736e-03   9.87196971e-04   1.06095389e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.61673688e-05   4.36463747e-03   4.28616739e-01   1.12886676e-04]
 [  5.20477078e-01   1.55984898e-03   2.17818008e-04   1.96113445e-03]
 [  5.43751992e-01   5.34993190e-03   2.86602642e-04   3.53291312e-04]
 [  0.

In [27]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.7725


In [28]:
%time

# Set learning parameters
lr = .8
y = .95
num_episodes = 10000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99999:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:])
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 13.4 µs


In [29]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.916
